Ray Tracer Manual

Imports first:

In [1]:
import sys
#add path
sys.path.append('/rds/general/user/le322/home/synthPy/')
import numpy as np
import matplotlib.pyplot as plt

import vtk
from vtk.util import numpy_support as vtk_np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'vtk'

To use the ray tracer, first create a 'domain', load the domain with an electron distribution, calculate electron gradients, initialise a beam, and solve!
\
\
The output can either be plotted, or passed further through ray optics for synthetic diagnostics

In [28]:
import solver.full_solver as s

# define some extent, the domain should be distributed as +extent to -extent, does not need to be cubic

extent_x = 5
extent_y = 5
extent_z = 10

n_cells = 5

x = np.linspace(-extent_x, extent_x, n_cells)
y = np.linspace(-extent_y, extent_y, n_cells)
z = np.linspace(-extent_z, extent_z, n_cells)

probing_extent = extent_z
probing_direction = 'z'

domain = s.ScalarDomain(x = x, y = y, z = z, extent= probing_extent, probing_direction = probing_direction) #create domain



Load some domain distribution

In [29]:
# load some external pvti


# import utils.handle_filetypes as load
# ne, dim, spacing = load.pvti_readin(filename)

# or load a test distribution

domain.test_exponential_cos()

Solve gradients, initialise beam, and solve

In [30]:
wl = 532e-9 #define laser wavelength

# initialise beam

Np = 1 #number of photons
divergence = 5e-5 #realistic divergence value
beam_size = extent_x #beam radius
ne_extent = probing_extent #so the beam knows where to initialise initial positions
beam_type = 'circular'


initial_rays = s.init_beam(Np = Np, beam_size = beam_size, divergence = divergence, ne_extent = ne_extent, beam_type = beam_type, probing_direction = probing_direction)

# solve ray trace

domain.calc_dndr(wl)

final_rays = domain.solve(initial_rays)

/rds/general/user/le322/home/anaconda3/envs/MAGPIE_venv/lib/python3.9/site-packages/numpy/lib/function_base.py:1238: RuntimeWarning: invalid value encountered in subtract
  out[tuple(slice1)] = (f[tuple(slice4)] - f[tuple(slice2)]) / (2. * ax_dx)


Pass through ray optics: For Shadowgaphy, Schlieren, or Refractometry:

In [32]:
import solver.rtm_solver as rtm

#in the diagnostic initialisation, details on the lens configurations, and detector dimensions can be specified
refractometer = rtm.RefractometerRays(final_rays)
refractometer.solve()
refractometer.histogram(bin_scale = 1, clear_mem = True)

sh=rtm.ShadowgraphyRays(final_rays)
sh.solve(displacement = 0)
sh.histogram(bin_scale = 1, clear_mem=True)

sh=rtm.SchlierenRays(final_rays)
sh.solve()
sh.histogram(bin_scale = 1, clear_mem=True)

#information accessed by .H(istogram) , e.g plt.imshow(refractometer.H)

Synthetic Interferometry - This requires some extra steps in the ray tracing to include information on the phase and electric field values

In [26]:
import solver.full_solver as s

extent_x = 5
extent_y = 5
extent_z = 10

n_cells = 2

x = np.linspace(-extent_x, extent_x, n_cells)
y = np.linspace(-extent_y, extent_y, n_cells)
z = np.linspace(-extent_z, extent_z, n_cells)

probing_extent = extent_z
probing_direction = 'z'

domain = s.ScalarDomain(x = x, y = y, z = z, extent= probing_extent, probing_direction = probing_direction, phaseshift = True) #specify phaseshift TRUE

# load some external pvti


# import utils.handle_filetypes as load
# ne, dim, spacing = load.pvti_readin(filename)

# or load a test distribution

domain.test_exponential_cos()

wl = 532e-9 #define laser wavelength

# initialise beam

Np = 1 #number of photons
divergence = 5e-5 #realistic divergence value
beam_size = extent_x #beam radius
ne_extent = probing_extent #so the beam knows where to initialise initial positions
beam_type = 'circular'


initial_rays = s.init_beam(Np = Np, beam_size = beam_size, divergence = divergence, ne_extent = ne_extent, beam_type = beam_type, probing_direction = probing_direction)

# solve ray trace

domain.calc_dndr(wl)

final_rays, E = domain.solve_with_E(initial_rays)


n_fringes = 10
deg = 10 #adjust number of fringes and tilt in degrees from the vertical exis
final_E = s.interfere_ref_beam(final_rays, E, n_fringes, deg)

pattern=rtm.InterferometerRays(final_rays, E = final_E)
pattern.solve(wl = wl)
pattern.interferogram(bin_scale = 1, clear_mem=True) #use interferogram instead of histogram

#get interferogram histogram with sh.H


KeyboardInterrupt: 